# 統計LODを色々試すnotebook
## プロパティパスを使ったSPQRQL、使わないSPARQL

SPARQL初心者の自分は、プロパティパスを使った場合エリアコードをキーに興味のある様々な値を複数のプロパティを使ってどのように取得すべきかわからないので、冗長ではあるけどエリアコードの変数を明示してプロパティパスを使わずサンプルを書いています。

In [1]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
# 新宿区の日本人の人口を求める（プロパティパス）
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o 
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja ;
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}


Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

In [5]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
# プロパティパスを使わないで書いてみる
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o 
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode ?areacode.
      ?areacode  ic:市区町村  "新宿区"@ja.
      ?s cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}


Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

# 統計LODサポートを使って興味のあるプロパティを検索し、統計LODのデータを取得する

## 保育園数や保育園入所待機児童数を取得してみます。
- [統計LODサポート](http://idease.info/lodstat/)に「保育」と入力して検索。
- 「保育所数」「保育所在所児数」「保育所入所待機児童数」などのプロパティが表示されます。この中からまず、保育所入所待機児童数を取得することにします。
- 統計LODサポートで検索された「保育所入所待機児童数」をクリックします。
- 保育所数のプロパティ（predicate）「http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/J250502」 が長いのでPREFIXを"ssds-measure-2016"のように設定します。
- 待機児童数（この場合RDFの目的語になる）を指す変数を「?taikijido」とします
- セルID　 保育所数の述語　　待機児童数; のRDFを記述
- セルID（省略形式）　エリアコード（PREFIXを利用）を指す述語 エリアコード（未知の変数）. を記述
- [共通語彙基盤](http://imi.go.jp/ns/core/Core232.html)のコア語彙「市区町村」を使って、エリアコード  市区町村 新宿区. を記述し、新宿区のエリアコードを取得します。
- SPARQLを実行すると新宿区の待機児童数が変数taikijidoとして取得されます。


In [6]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
#新宿区の待機児童数
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?taikijido
where{
    ?s ssds-measure-2016:J250502 ?taikijido;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:市区町村  "新宿区"@ja.
}


Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['taikijido']},
 'results': {'bindings': [{'taikijido': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                         'type': 'typed-literal',
                                         'value': '176'}}]}}

## UNIONを使って「新宿区」と「文京区」のOR条件で取得する

In [9]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?o
where{
        {
        ?s ssds-measure-2016:J2503 ?o;
        cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja. 
            } union {
        ?s ssds-measure-2016:J2503 ?o;
        cd-dimension:standardAreaCode / ic:市区町村 "文京区"@ja. 
        }
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

o
36
28


## 都道府県名のプロパティで東京都のエリアコードを取得し、東京都の待機児童数を取得

In [36]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?taikijido ?shiku
where{
    ?s ssds-measure-2016:J250502 ?taikijido;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?shiku.
}
limit 10

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

taikijido,shiku
181,武蔵野市
181,墨田区
181,府中市
47,狛江市
184,西東京市
19,青梅市
96,文京区
35,あきる野市
257,町田市
270,豊島区


## 区ごとの「保育所在所児童数」「保育所数」を組み合わせてみます。
プロパティパスで逆パス等を使うとまとめて記述できそうだけど、、、、自分は書けないのでとりあえずシンプルに書いてみます。

In [7]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?shiku ?hoikusyo  ?zaisyo ?taiki
where{
    ?s1 ssds-measure-2016:J250502 ?taiki;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?shiku.
    ?s2 ssds-measure-2016:J2503 ?hoikusyo;
        cd-dimension:standardAreaCode ?areacode.
    ?s3 ssds-measure-2016:J2506 ?zaisyo;
        cd-dimension:standardAreaCode ?areacode.
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

shiku,hoikusyo,zaisyo,taiki
昭島市,19,2517,46
板橋区,93,8477,417
青梅市,32,3322,19
豊島区,31,3041,270
利島村,1,24,0
練馬区,89,8814,578
御蔵島村,0,0,0
小平市,18,1956,174
多摩市,19,2324,75
江戸川区,83,9544,192
